In [1]:
# imports
import pandas as pd
import json
import re

Handling file conversions for SQL

In [2]:
# converting from sql 
def convert_from_sql(sql_file_path, result_type):
    # read file
    with open(sql_file_path, 'r') as file:
        sql_content = file.read()
    
    # extract column names
    columns_match = re.search(r'INSERT INTO \w+ \((.*?)\)', sql_content)
    if columns_match:
        columns = [col.strip() for col in columns_match.group(1).split(',')]
    else:
        raise ValueError("Could not find column names in SQL file")
    
    # extract values
    values_match = re.findall(r'VALUES\s*(\(.*?\))', sql_content)
    if not values_match:
        raise ValueError("Could not find VALUES in SQL file")
    
    # parse values into rows
    rows = []
    for value_group in values_match:
        # Remove parentheses and split by comma
        values = [v.strip().strip("'\"") for v in value_group.strip('()').split(',')]
        rows.append(values)
    
    # create df and convert
    df = pd.DataFrame(rows, columns=columns)
    if result_type.lower() == "json":
        df.to_json("output_data/", orient='records')
    elif result_type.lower() == "csv":
        df.to_csv("output_data/", index=False)
    else:
        raise ValueError("Desired output type wasn't JSON or CSV")

Handling file conversions for JSON

In [3]:
# converting from json
def convert_from_json(json_file_path, result_type):
    # read file
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)
    
    # create df and convert
    df = pd.DataFrame(json_data)
    if result_type.lower() == "sql":
        sql_query = df.to_sql('table_name', if_exists='replace', index=False)
        with open("output_data/", 'w') as file:
            file.write(sql_query)
    elif result_type.lower() == "csv":
        df.to_csv("output_data/", index=False)
    else:
        raise ValueError("Desired output type wasn't SQL or CSV")


Handling file conversions for CSV

In [4]:
# converting from csv
def convert_from_csv(csv_file_path, result_type):
    # read file and convert
    df = pd.read_csv(csv_file_path)
    if result_type.lower() == "json":
        df.to_json("output_data/", orient='records')
    elif result_type.lower() == "sql":
        sql_query = df.to_sql('table_name', if_exists='replace', index=False)
        with open("output_data/", 'w') as file:
            file.write(sql_query)
    else:
        raise ValueError("Desired output type wasn't JSON or SQL")
